In [10]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from pylab import *
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([transforms.Grayscale(3),
                                 transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.Grayscale(3),
                               transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "/content/drive/MyDrive/ml_wildfire/processed_data"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([4, 3, 32, 32])
torch.Size([4])


In [7]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.9964 Acc: 0.5450
val Loss: 1.0547 Acc: 0.5467

Epoch 1/24
----------
train Loss: 0.9804 Acc: 0.5124
val Loss: 1.0229 Acc: 0.4547

Epoch 2/24
----------
train Loss: 0.9626 Acc: 0.5206
val Loss: 0.8276 Acc: 0.5609

Epoch 3/24
----------
train Loss: 0.9948 Acc: 0.5227
val Loss: 0.7459 Acc: 0.5439

Epoch 4/24
----------
train Loss: 0.9522 Acc: 0.5245
val Loss: 0.7372 Acc: 0.4249

Epoch 5/24
----------
train Loss: 0.9324 Acc: 0.5305
val Loss: 1.2971 Acc: 0.4533

Epoch 6/24
----------
train Loss: 0.9316 Acc: 0.5411
val Loss: 1.1444 Acc: 0.5524

Epoch 7/24
----------
train Loss: 0.7146 Acc: 0.5514
val Loss: 0.6921 Acc: 0.5510

Epoch 8/24
----------
train Loss: 0.7071 Acc: 0.5510
val Loss: 0.6833 Acc: 0.5510

Epoch 9/24
----------
train Loss: 0.6969 Acc: 0.5638
val Loss: 0.7930 Acc: 0.5567

Epoch 10/24
----------
train Loss: 0.6981 Acc: 0.5556
val Loss: 0.6851 Acc: 0.5453

Epoch 11/24
----------
train Loss: 0.6958 Acc: 0.5599
val Loss: 0.6805 Acc: 0.5708

Ep

3 Months Data

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "/content/drive/MyDrive/ml_wildfire/processed_3datapng"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([32, 3, 32, 32])
torch.Size([32])


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6895 Acc: 0.5472
val Loss: 4741750.2679 Acc: 0.5434

Epoch 1/24
----------
train Loss: 0.6898 Acc: 0.5472
val Loss: 25385808.9680 Acc: 0.5434

Epoch 2/24
----------
train Loss: 0.6903 Acc: 0.5472
val Loss: 2959523.0285 Acc: 0.5434

Epoch 3/24
----------
train Loss: 0.6893 Acc: 0.5472
val Loss: 7290107.1758 Acc: 0.5434

Epoch 4/24
----------
train Loss: 0.6911 Acc: 0.5472
val Loss: 2111468.2618 Acc: 0.5434

Epoch 5/24
----------
train Loss: 0.6911 Acc: 0.5340
val Loss: 1731609.9924 Acc: 0.5434

Epoch 6/24
----------
train Loss: 0.6913 Acc: 0.5472
val Loss: 730219.7197 Acc: 0.5434

Epoch 7/24
----------
train Loss: 0.6889 Acc: 0.5472
val Loss: 420753.8455 Acc: 0.5434

Epoch 8/24
----------
train Loss: 0.6891 Acc: 0.5472
val Loss: 1584864.1892 Acc: 0.5434

Epoch 9/24
----------
train Loss: 0.6890 Acc: 0.5472
val Loss: 1007592.2578 Acc: 0.5434

Epoch 10/24
----------
train Loss: 0.6889 Acc: 0.5472
val Loss: 10612.2823 Acc: 0.5434

Epoch 11/24
----------
